In [ ]:

!wget http://images.cocodataset.org/zips/train2017.zip
!unzip -q train2017.zip -d coco/
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d coco/

import json, os
from tqdm import tqdm
import numpy as np
import string
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from tensorflow.keras.models import Model

--2025-08-05 06:31:03--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.217.234, 54.231.225.185, 52.217.198.1, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.217.234|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  60.2MB/s    in 8m 10s  

2025-08-05 06:39:14 (37.6 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2025-08-05 06:42:33--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.220.97, 3.5.27.189, 3.5.29.64, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.220.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_t

In [ ]:
import os
import json
import string
import pickle
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model


ann_path = "/content/coco/annotations/captions_train2017.json"
img_dir = "/content/coco/train2017"


print("Loading annotations...")
with open(ann_path, 'r') as f:
    annotations = json.load(f)


image_id_to_file = {img['id']: img['file_name'] for img in annotations['images']}


descriptions = {}
for ann in annotations['annotations']:
    img_id = ann['image_id']
    caption = ann['caption'].lower().translate(str.maketrans('', '', string.punctuation))
    fname = image_id_to_file[img_id]
    descriptions.setdefault(fname, []).append("startseq " + caption + " endseq")

print("Total images with captions:", len(descriptions))


print("Loading InceptionV3 model...")
base_model = InceptionV3(weights='imagenet')
cnn_model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)


batch_size = 128
features = {}
output_file = "/content/features.pkl"
save_every = 1000
image_files = list(descriptions.keys())

print(f"Starting feature extraction for {len(image_files)} images...")


for i in tqdm(range(0, len(image_files), batch_size)):
    batch_files = image_files[i:i+batch_size]
    batch_images = []
    valid_files = []

    for fname in batch_files:
        path = os.path.join(img_dir, fname)
        if not os.path.exists(path):
            continue
        img = image.load_img(path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        batch_images.append(x)
        valid_files.append(fname)

    if len(batch_images) == 0:
        continue

    batch_images = np.array(batch_images)


    batch_features = cnn_model.predict(batch_images, verbose=0)


    for j, fname in enumerate(valid_files):
        features[fname] = batch_features[j]


    if (i + batch_size) % save_every == 0:
        with open(output_file, "wb") as f:
            pickle.dump(features, f)
        print(f"Checkpoint saved: {len(features)} images processed")


with open(output_file, "wb") as f:
    pickle.dump(features, f)

print(f"Feature extraction complete! Total images processed: {len(features)}")
print(f"Features saved to {output_file}")

Loading annotations...
Total images with captions: 118287
Loading InceptionV3 model...
Starting feature extraction for 118287 images...


 14%|█▎        | 125/925 [03:58<24:33,  1.84s/it]

Checkpoint saved: 16000 images processed


 27%|██▋       | 250/925 [07:31<19:36,  1.74s/it]

Checkpoint saved: 32000 images processed


...

Checkpoint saved: 96000 images processed


 95%|█████████▍| 875/925 [25:18<03:02,  3.65s/it]

Checkpoint saved: 112000 images processed


100%|██████████| 925/925 [26:54<00:00,  1.75s/it]


Feature extraction complete! Total images processed: 118287
Features saved to /content/features.pkl
